<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#240b36;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px; color:white; text-align: center"> WELCOME TO MY NOTEBOOK </p>
</div>

<h1 style="color:#E83500; text-align: center"> Title: Brain Tumor Classification with custom Neural Network </font></h1>

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="http://alsani.me/wp-content/uploads/2023/11/MRI-animinated.gif" alt="Centered Image" style="display: block; margin: auto;">
</div>


<a id="home"></a>

### Table of contents:

* [Introduction and brief introduction of data](#1)
* [1. Import Necessary Library](#2)
* [2. Data Loading and Preprocessing](#3)
* [3. DenseNet201 function Calling](#4)
* [5. Now Plot the Training Loss, Validation Loss](#5)
* [6. Now Predictions for random image](#6)

<a id="1"></a>
<div style= "border-radius:10px; border:#787878 solid; padding: 15px; background-color: #E7E5E4; font-size:100%; text-align:left">

<h2 style="color:#E83500"> ▶️ Introduction and brief introduction of data</font></h2>

**Introduction**

A Brain tumor is considered as one of the aggressive diseases, among children and adults. Brain tumors account for 85 to 90 percent of all primary Central Nervous System(CNS) tumors. Every year, around 11,700 people are diagnosed with a brain tumor. The 5-year survival rate for people with a cancerous brain or CNS tumor is approximately 34 percent for men and36 percent for women. Brain Tumors are classified as: Benign Tumor, Malignant Tumor, Pituitary Tumor, etc. Proper treatment, planning, and accurate diagnostics should be implemented to improve the life expectancy of the patients. The best technique to detect brain tumors is Magnetic Resonance Imaging (MRI). A huge amount of image data is generated through the scans. These images are examined by the radiologist. A manual examination can be error-prone due to the level of complexities involved in brain tumors and their properties.

Application of automated classification techniques using Machine Learning(ML) and Artificial Intelligence(AI)has consistently shown higher accuracy than manual classification. Hence, proposing a system performing detection and classification by using Deep Learning Algorithms using ConvolutionNeural Network (CNN), Artificial Neural Network (ANN), and TransferLearning (TL) would be helpful to doctors all around the world.


**Summary**

Brain Tumors are complex. There are a lot of abnormalities in the sizes and location of the brain tumor(s). This makes it really difficult for complete understanding of the nature of the tumor. Also, a professional Neurosurgeon is required for MRI analysis. Often times in developing countries the lack of skillful doctors and lack of knowledge about tumors makes it really challenging and time-consuming to generate reports from MRI’. So an automated system on Cloud can solve this problem.

**Read More About Dataset:** [Click](https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri)

**The key folders in the dataset include:**

1. GLIOMA TUMOR
1. MENINGIOMA TUMOR
1. NO TUMOR
1. PITUITARY TUMOR



Remote source:https://github.com/sartajbhuvaji/brain-tumor-classification-dataset
The folder contains MRI data. The images are already split into Training and Testing folders.
Each folder has more four subfolders. These folders have MRIs of respective tumor classes.

<a id="2"></a>

<p style="font-family:FRIZON; font-weight:normal; letter-spacing: 1px; color:#004C88; font-size:200%; text-align:left;padding: 10px; border-bottom: 3px solid #0081E5;">1. Import Necessary Library</p>

<a class="btn" href="#home">🏠 Table of Contents</a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping , ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical,plot_model

import numpy as np
import os
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings("ignore")
%matplotlib inline

<a id="3"></a>

<p style="font-family:FRIZON; font-weight:normal; letter-spacing: 1px; color:#004C88; font-size:200%; text-align:left;padding: 10px; border-bottom: 3px solid #0081E5;">2. Data Loading and Preprocessing</p>

<a class="btn" href="#home">🏠 Table of Contents</a>

In [ ]:
train_datasets = "../input/brain-tumor-classification-mri/Training/"
validation_datasets = "../input/brain-tumor-classification-mri/Testing/"

In [ ]:
batch_size = 64
image_size = 224

In [ ]:

def prepare_the_datasets(train_datasets, validation_datasets, batch_size, image_size):

    train_datasets_generator = ImageDataGenerator(rescale=1./255,
                                                  shear_range = 0.2, 
                                                  zoom_range = 0.2, 
                                                  horizontal_flip = True, 
                                                  fill_mode = "nearest")


    validation_datasets_generator = ImageDataGenerator(rescale=1.0/255)


    train_datasets_generator_data = train_datasets_generator.flow_from_directory(
        batch_size = batch_size,
        directory = train_datasets,
        shuffle = True,
        target_size = (image_size, image_size),
        class_mode = "categorical"
    )

    validation_datasets_generator_data = validation_datasets_generator.flow_from_directory(
        batch_size = batch_size,
        directory = validation_datasets,
        shuffle = True,
        target_size = (image_size, image_size),
        class_mode = "categorical"
    )


    return train_datasets_generator_data, validation_datasets_generator_data

In [ ]:
train_data , validation_data = prepare_the_datasets(train_datasets, validation_datasets, batch_size, image_size)

<a id="4"></a>

<p style="font-family:FRIZON; font-weight:normal; letter-spacing: 1px; color:#004C88; font-size:200%; text-align:left;padding: 10px; border-bottom: 3px solid #0081E5;">3. DenseNet201 function Calling </p>

<a class="btn" href="#home">🏠 Table of Contents</a>

<h3 align="left"><font color='#E83500'>💡 Note:</font></h3>
    
Each Keras Application expects a specific kind of input preprocessing. For DenseNet, call tf.keras.applications.densenet.preprocess_input on your inputs before passing them to the model.

**Arguments**

**1. include_top:** whether to include the fully-connected layer at the top of the network.

**1. weights:** one of None (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.

**1. input_tensor:** optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.

**1. input_shape:** optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) (with 'channels_last' data format) or (3, 224, 224) (with 'channels_first' data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 32. E.g. (200, 200, 3) would be one valid value.

**1. pooling:** Optional pooling mode for feature extraction when include_top is False.
> **None means** that the output of the model will be the 4D tensor output of the last convolutional block.

> **avg means** that global average pooling will be applied to the output of the last convolutional block, and thus the output of the model will be a 2D tensor.

> **max means** that global max pooling will be applied.

**1. classes:** optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified.

**1. classifier_activation:** A str or callable. The activation function to use on the "top" layer. Ignored unless include_top=True. 
Set classifier_activation=None to return the logits of the "top" layer. When loading pretrained weights, classifier_activation can only be None or "softmax".

From [Keras Application website](https://keras.io/api/applications/densenet/#densenet201-function)

In [ ]:
densenet= tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    pooling=None,
    input_shape=(224,224,3)
)

In [ ]:
for layer in densenet.layers:
    layer.trainable= False

In [ ]:
""" Creates the top or head of the model that will be placed on top of the layers """

def addTopModel(bottom_model, num_class, D=256):
    top_model= bottom_model.output
    top_model= Flatten(name="flatten") (top_model)
    top_model= Dense(D, activation="relu") (top_model)
    top_model= Dropout(0.3) (top_model)
    top_model= Dense(num_class, activation="softmax") (top_model)
    return top_model

In [ ]:
num_classes= 4
Fc_Head= addTopModel(densenet, num_classes)
model= Model(inputs= densenet.input, outputs= Fc_Head)

In [ ]:
checkpoint= ModelCheckpoint("/Trained Models/densenet101.h5",
                            monitor= "val_loss",
                            mode= "min",
                            save_best_only= True,
                            verbose=1)

earlystop= EarlyStopping(monitor= "val_loss",
                         min_delta= 0,
                         patience= 3,
                         verbose=1,
                         restore_best_weights= True)


reduce_lr= ReduceLROnPlateau(monitor= "val_loss",
                         factor=0.2,   
                         patience= 5,
                         verbose=1,
                         min_delta= 0.000001)


callbacks=[earlystop, checkpoint, reduce_lr]

In [ ]:
model.compile(loss= 'categorical_crossentropy',
              optimizer= RMSprop(learning_rate=0.00001),
              metrics= ['accuracy'])

In [ ]:
nb_train_samples= 2870
nb_validation_samples= 394
epochs= 20
batch_size= 128

history= model.fit(train_data,
                   steps_per_epoch= nb_train_samples//batch_size,
                   epochs= epochs,
                   callbacks=callbacks,
                   validation_data= validation_data,
                   validation_steps= nb_validation_samples//batch_size)

In [ ]:
model_evaluation= model.evaluate(validation_data, batch_size = batch_size)

<a id="5"></a>

<p style="font-family:FRIZON; font-weight:normal; letter-spacing: 1px; color:#004C88; font-size:200%; text-align:left;padding: 10px; border-bottom: 3px solid #0081E5;">5. Now Plot the Training Loss, Validation Loss, Training Accuracy, Validation Accuracy</p>

<a class="btn" href="#home">🏠 Table of Contents</a>

In [ ]:
import numpy as np

def plot_training_curves(history):
    loss= np.array(history.history['loss'])
    val_loss= np.array(history.history['val_loss'])
    
    accuracy= np.array(history.history['accuracy'])
    val_accuracy= np.array(history.history['val_accuracy'])
    
    epochs= range(len(history.history['loss']))
    
    fig, (ax1, ax2)= plt.subplots(1,2,figsize=(10,3))
    
    #plot loss
    ax1.plot(epochs, loss, label='traing_loss', marker='o')
    
    ax1.plot(epochs, val_loss, label='val_loss', marker='o')
    
    ax1.fill_between(epochs,loss, val_loss, where=(loss>val_loss),color='C0',alpha=0.3,interpolate=True)
    ax1.fill_between(epochs,loss, val_loss, where=(loss<val_loss),color='C1',alpha=0.3,interpolate=True)
    
    ax1.set_title('Loss(Lower Means Better)',fontsize= 16)
    ax1.set_xlabel('Epochs', fontsize=10)
    
    ax1.legend()
    
    #plot Accuracy
    ax2.plot(epochs, accuracy, label='traing_accuracy', marker='o')
    
    ax2.plot(epochs, val_accuracy, label='val_accuracy', marker='o')
    
    ax2.fill_between(epochs,accuracy, val_accuracy, where=(accuracy>val_accuracy),color='C0',alpha=0.3,interpolate=True)
    ax2.fill_between(epochs,accuracy, val_accuracy, where=(accuracy<val_accuracy),color='C1',alpha=0.3,interpolate=True)
    
    ax2.set_title('Accuracy(Higher Means Better)',fontsize= 16)
    ax2.set_xlabel('Epochs', fontsize=10)
    
    ax2.legend()

In [ ]:
plot_training_curves(history)

<a id="6"></a>

<p style="font-family:FRIZON; font-weight:normal; letter-spacing: 1px; color:#004C88; font-size:200%; text-align:left;padding: 10px; border-bottom: 3px solid #0081E5;">6. Now Predictions for random image
</p>

<a class="btn" href="#home">🏠 Table of Contents</a>

In [ ]:
from tensorflow.keras.applications.imagenet_utils import preprocess_input
img= image.load_img("../input/brain-tumor-classification-mri/Testing/no_tumor/image(10).jpg",target_size=(224,224))

x= image.img_to_array(img)
x=x/255
x= np.expand_dims(x, axis=0)
img_data=preprocess_input(x)
img_data.shape
preds= model.predict(x)
preds= np.argmax(preds, axis=1)
if preds==1:
    preds="The image is glima Tumor"
elif preds==2:
    preds="The image is meningioma Tumor"
elif preds==3:
    preds="The image is No Tumor"
else:
    preds="The image is pituitary tumor"
print(preds)

In [ ]:
# Model Saving on directory

model.save("../working/densenet101.h5")
y_pred= model.predict(validation_data)
print(y_pred)
y_pred= np.argmax(y_pred, axis=1)


# Conclusion
**Please feel free to ask in the comment section if you have any confusion or questions.**

**Here are some of the contributions I've made on Kaggle:**
1. [Pie Charts in Python](https://www.kaggle.com/code/alsaniipe/pie-charts-in-python)
1. [Scatter plots with Plotly Express](https://www.kaggle.com/code/alsaniipe/scatter-plots-with-plotly-express)
1. [X-ray Image Classification using Transfer Learning](https://www.kaggle.com/code/alsaniipe/x-ray-image-classification-using-transfer-learning)
1. [Flowers Classification by Using VGG16 Model 🎉🎉](https://www.kaggle.com/code/alsaniipe/flowers-classification-by-using-vgg16-model)
1. [Car Brand Prediction's by Using ResNet50 Model](https://www.kaggle.com/code/alsaniipe/car-brand-prediction-s-by-using-resnet50-model)
1. [Image Preprocessing-Morpological Analysis & Kernel](https://www.kaggle.com/code/alsaniipe/image-preprocessing-morpological-analysis-kernel)
1. [Image Similarity Index (SSIM analysis )](https://www.kaggle.com/code/alsaniipe/image-similarity-index-ssim-analysis)
1. [Image Preprocessing- Image Transformation & OpenCV](https://www.kaggle.com/code/alsaniipe/image-preprocessing-image-transformation-opencv)